# Chainerの学習済み物体検出モデルをホスティング

## 概要
このノートブックでは、Chainerの学習済み物体検出モデルをダウンロードしてホスティングします。物体検出のアルゴリズムはいくつかありますが、ここではSSD (Single Shot Multibox Detector) を利用します。Chainer公式の学習済みモデルは以下からダウンロードすることができます。

https://github.com/chainer/chainercv/tree/master/examples/ssd

### (注意点)
ChainerCVでは、特定のDeep Neural Networks(SSD含む)を構築する関数に、学習済みモデルをロードする機能が提供されているため、本来は*事前の物体検出モデルのダウンロードは不要*です。今回は、学習済みモデルをSageMakerに取り込む方法を一通り体験するために、事前にダウンロードして、別途ロードするという手順を行っています。

## 学習済みモデルのダウンロードとS3へのアップロード

上記のURLでは、SSD300とSSD512のモデルが提供されており、今回はSSD300を利用します。300や512は入力画像のサイズを表します。一般に512のほうが精度が良いですが、推論に多くの計算を必要とします。

SSD300のモデルをダウンロードするために以下を実行します。ダウンロードされたモデルは、このノートブックインスタンスの`/tmp/ssd_model.npz`に保存されます。SageMakerでモデルをホスティングするためには、tar.gz形式にしてS3にアップロードする必要があります。`model.tar.gz`に変換した後、SageMaker Python SDKの`upload_data`を利用してS3にアップロードします。アップロードされる先は、`s3://sagemaker-{リージョン名}-{12桁アカウントID}/notebook/chainercv_ssd/model.tar.gz`になります。

In [ ]:
import os
import shutil
import tarfile
import urllib.request

# Setup
from sagemaker import get_execution_role
import sagemaker
sagemaker_session = sagemaker.Session()

# This role retrieves the SageMaker-compatible role used by this Notebook Instance.
role = get_execution_role()

# Download the model weights.
try:
    url = 'https://chainercv-models.preferred.jp/ssd300_voc0712_trained_2017_08_08.npz'
    urllib.request.urlretrieve (url, '/tmp/ssd_model.npz')

# Tar and compress the model.
    with tarfile.open('/tmp/model.tar.gz', "w:gz") as tar:
         tar.add('/tmp/ssd_model.npz', arcname='ssd_model.npz')

# Upload the model. The `ChainerModel` will use `uploaded_data` to download this model.

    uploaded_model = sagemaker_session.upload_data(path='/tmp/model.tar.gz', 
                                                   key_prefix='notebook/chainercv_ssd')
finally:
    os.remove('/tmp/model.tar.gz')

## ホスティング用のスクリプトの作成

SageMakerでホスティングするためには、モデルの利用方法（どういうモデルを読み込むか、前処理をいれるかなど）を決めるPythonスクリプトが必要になります。Chainerの場合は、シリアライズされたモデルはネットワークの重みだけを含んでいるため、シンボルを定義して、そこに重みの値をロードするという処理が必要になります。

同梱されている`chainercv_ssd.py`が、そのためのスクリプトです。`chainercv_ssd.py`には、`model_fn`と`predict_fn`の関数が定義されていることがわかります。以下は、ホスティングのために実装する関数の一覧です。このうち `model_fn` は必ず実装しなければなりません。


### ホスティング用関数一覧

* **`model_fn(model_dir)`**: この関数は`model_dir`に保存されているモデルをロードする関数です。上述したように、シンボルの定義を行ってからロードします。シンボルの定義にはChainerCVを利用します。

* **`input_fn(input_data, content_type)`**: この関数は、推論リクエストを受け付けたときに、推論用のデータ`input_data`に対する前処理を書く関数です。`content_type`を同時に受け取ることができるので、`content_type`に応じて条件分岐を作成し、異なる前処理を実装することができます。
  
* **`predict_fn(input_data, model)`**: この関数は `input_fn` で前処理されてreturnされた値を`input_data`として受け取り、`model_fn`でロードした`model`で推論するコードを書く関数です。 
  
* **`output_fn(prediction, accept)`**: この関数は `predict_fn`のreturnした値`prediction`を後処理するための関数です。`accept`に応じて処理を変更することもできます。

### 関数の流れ
上記では文章で書きましたが、擬似的なコードで示すと、関数の実行順はこのような流れになります。
```python
# Load a model from file system
model = model_fn(model_dir)

# Deserialize the Invoke request body into an object we can perform prediction on
input_object = input_fn(request_body, request_content_type)

# Perform prediction on the deserialized object, with the loaded model
prediction = predict_fn(input_object, model)

# Serialize the prediction result into the desired response content type
output = output_fn(prediction, response_content_type)
```

## モデルをホスティング

S3にモデルをアップロードして、ホスティング用のスクリプトがそろったら、SageMakerにそのモデルを登録します。Chainerの場合は`ChainerModel`を利用します。このとき、アップロードしたモデルと、スクリプトを指定する必要があります。もしモデルを登録したら`deploy`でエンドポイントを作成します。

In [ ]:
from sagemaker.chainer.model import ChainerModel
from sagemaker.utils import sagemaker_timestamp

model = ChainerModel(model_data=uploaded_model, role=role, entry_point='chainercv_ssd.py')

endpoint_name = 'chainer-ssd-{}'.format(sagemaker_timestamp())

predictor = model.deploy(instance_type='ml.m4.xlarge', initial_instance_count=1, endpoint_name=endpoint_name)

## ホストしたモデルに対して推論リクエストを送信

モデルをホストできたら`predictor`を利用して画像を送ります。`predictor.predict(image)`でリクエストを送ることができます。結果が、`predict_fn`で定義したように、bounding box, label, score の順で返ってくることを確認しましょう。

In [ ]:
import chainercv
import numpy as np
from matplotlib import pyplot as plot

image = chainercv.utils.read_image('images/poodle.jpg', color=True)
image = np.ascontiguousarray(image, dtype=np.uint8)

In [ ]:
bbox, label, score = predictor.predict(image)
print('bounding box: {}\nlabel: {}\nscore: {}'.format(bbox, label, score))

ChainerCVを利用すると、簡単に画像の上にbounding box、label、scoreをのせることができます。

In [ ]:
%matplotlib inline
from chainercv.visualizations import vis_bbox
from chainercv.datasets import voc_bbox_label_names
import matplotlib.pyplot as plt
vis_bbox(image, bbox, label, score, label_names=voc_bbox_label_names)
plt.show()

## Amazon SageMaker Neoによるモデルのコンパイル

**2019.02.28時点では、Neoが物体検出モデルのコンパイルに完全対応しておらず、以降のコードでコンパイルを完了することはできません。**

Amazon SageMakerでは、モデルを最適化するためのNeoというサービスを提供しています。モデルの最適化によって、推論速度を向上したり、メモリ消費量を低減したりできるかもしれません。

### ONNXモデルへの変換

onnx-chainerを利用して、さきほど`/tmp/ssd_model.npz`にダウンロードしたChainerのモデルを、ONNXモデル`/tmp/ssd.onnx`に変換します。その前にSageMakerにインストールされているonnx-chainerを再インストールして新しくします。

In [ ]:
!pip uninstall -y onnx-chainer
!pip install --no-cache-dir onnx-chainer

インストールが終わると、ChainerCVを利用して、ダウンロードしたモデルを読み込み、`onnx_chainer.export()`を利用して、`/tmp/onnx/model/ssd.onnx`に変換ファイルを保存します。その後、`model.tar.gz`にしてからS3にアップロードします。

In [ ]:
import onnx_chainer
import chainer
import numpy as np
from chainercv.links import SSD300
from chainercv.datasets import voc_bbox_label_names

from sagemaker import get_execution_role
import sagemaker
sagemaker_session = sagemaker.Session()

# This role retrieves the SageMaker-compatible role used by this Notebook Instance.
role = get_execution_role()

path = '/tmp/ssd_model.npz'
model = SSD300(n_fg_class=len(voc_bbox_label_names), pretrained_model=path)

# Prepare dummy data
x = np.zeros((1, 3, 300, 300), dtype=np.float32)

# Put Chainer into inference mode
chainer.config.train = False

# Convert the model to ONNX format
import os
os.makedirs('/tmp/onnx/model', exist_ok=True)
onnx_model = onnx_chainer.export(model, x, filename='/tmp/onnx/model/ssd.onnx')

import tarfile
archive = tarfile.open('/tmp/onnx/model.tar.gz', mode='w:gz')
archive.add('/tmp/onnx/model/', arcname = "model")
archive.close()

uploaded_onnx = sagemaker_session.upload_data(path='/tmp/onnx/model.tar.gz', 
                                                   key_prefix='notebook/onnx')
print("ONNX model is uploaded to {}".format(uploaded_onnx))

## モデルのコンパイル

モデルをアップロードしたら、Neoを利用してコンパイルします。

In [ ]:
from sagemaker.model import Model
onnx_model = Model(model_data = uploaded_onnx,
                   image = None,
                   sagemaker_session=sagemaker_session,
                   role = role)

from datetime import datetime
compiled_model = onnx_model.compile(target_instance_family='ml_c5', 
                                     input_shape={'Input_0': [1,3,300,300]},
                                     output_path = 's3://'+sagemaker_session.default_bucket() + '/compiled-model/ssd/',
                                     role = role,
                                     job_name ="compiled-from-chainer-" + datetime.now().strftime("%Y-%m-%d-%H-%M-%S"),
                                     framework='onnx')